# Notebook for testing connection to Tensorflow-Serving

## Installing libraries
- grpc
- tensorflow-serving-api\
- keras image helper

In [ ]:
!pip install grpcio==1.49.1 tensorflow-serving-api==2.17.0

In [ ]:
!pip install keras-image-helper

## Load Libraries

In [22]:
import grpc
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
import numpy as np

## Setting up channel

In [2]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

## Preproccessing Image for Prediction

In [3]:
from keras_image_helper import create_preprocessor

In [4]:
preprocessor = create_preprocessor('xception', target_size=(224, 224))

In [5]:
#url = 'https://github.com/juangrau/ML-Capstone-Project-1/blob/main/AnnualCrop_1000.jpg?raw=true'
url = 'https://tensorflow.org/images/blogs/serving/cat.jpg'
X = preprocessor.from_url(url)

In [28]:
X.shape

(1, 224, 224, 3)

In [6]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

## Setup gRPC Request

In [7]:
pb_request = predict_pb2.PredictRequest()

'''
Now we use the information we got from the tensorflow cli:

serving_default -> signature name
inputs  -> Input
output_0 -> output
'''


pb_request.model_spec.name = 'resnet'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_1'].CopyFrom(np_to_protobuf(X))


## Execute request to TF Serving

In [8]:
pb_response = stub.Predict(pb_request, timeout=30.0)

In [21]:
result = pb_response.outputs['activation_49'].float_val

In [23]:
np.argmax(result)

971

In [27]:
print('Prediction Class Index: {}, with a value of {}'.format(np.argmax(result), result[np.argmax(result)]))

Prediction Class Index: 971, with a value of 0.13512633740901947
